In [10]:

import pandas as pd
import plotly.express as px

df = pd.read_csv("nuclear_data/levelised_cost_of_electricity.csv").drop(index=0)
df = df.rename(columns={
    'Capital costs': 'Capital',
    'O&M': 'OM',
    'Fuel (th)': 'Fuel_th',
    'Fuel (el)': 'Fuel_el',
    'Carbon': 'Carbon',
    'CHP heat revenues': 'CHP_revenue',
    'LCOE': 'LCOE',
    'Category': 'Category',
    'Country': 'Country'
})
df['LCOE'] = pd.to_numeric(df['LCOE'], errors='coerce')
df = df.dropna(subset=['Category', 'LCOE'])

nuclear_mean = df[df['Category'] == 'Nuclear']['LCOE'].mean()
competitors = ['Wind', 'Solar', 'Gas', 'Coal']
savings = {
    comp: ((df[df['Category'] == comp]['LCOE'].mean() - nuclear_mean) / 
           df[df['Category'] == comp]['LCOE'].mean() * 100)
    for comp in competitors
}

def create_hover_text(tech, nuclear_mean):
    tech_mean = df[df['Category'] == tech]['LCOE'].mean()
    abs_diff = tech_mean - nuclear_mean
    return f"Absolute Difference: ${abs_diff:.1f}/MWh"

savings_df = pd.DataFrame({
    'Technology': list(savings.keys()),
    'Savings (%)': list(savings.values()),
    'Hover Text': [create_hover_text(tech, nuclear_mean) for tech in savings.keys()]
})

fig = px.bar(savings_df, 
             x='Technology', 
             y='Savings (%)',
             text='Savings (%)',
             color='Technology',
             color_discrete_map={
            'Coal': '#EF553B',  
            'Gas': '#AB63FA', 
            'Solar': '#FFA15A',
            'Wind': '#00CC96'
        },

             title='<b>Nuclear Cost Advantage Over Other Forms Of Energy Production</b>',
             hover_data={'Technology': False, 'Savings (%)': False, 'Hover Text': True},
             labels={'Savings (%)': 'Percent Cheaper Than Competitor'})

fig.update_traces(
    texttemplate='%{text:.1f}%', 
    textposition='outside',
    marker_line_color='white',
    marker_line_width=1.5,
    opacity=0.8,
    hovertemplate='%{customdata[0]}<extra></extra>' 
)

fig.update_layout(
    yaxis_title="Percent Cheaper Than Competitor",
    xaxis_title="",
    hoverlabel=dict(
        bgcolor="white",
        font_size=15,
        font_family="Arial"
    ),
    showlegend=False,
    plot_bgcolor='rgba(0,0,0,0)',
    yaxis=dict(
        gridcolor='lightgray',
        zerolinecolor='black',
        zerolinewidth=2
    ),
    hovermode="closest",
    title_x=0.5,
    height=600,
    width=800,
    margin=dict(t=100, b=80, l=40, r=40)
)

fig.add_hline(y=0, line_width=2, line_color='black')

fig.show()